# Gaussian Process and Machine Learning

In [1]:
using ColorSchemes
using Optim
using Plots
using Printf
using Statistics
using LaTeXStrings

In [ ]:
include("gp.jl")

## Sampling from 1D Gaussian Process

In [ ]:
xs = collect(range(-5, stop = 5, length = 100))

gp = GaussianProcess(GaussianKernel(1, 1))
Plots.plot(xs, rand(dist(gp, xs), 5), 
    label = "", xlabel = "x", ylabel = "f",
    linewidth = 2,
    title = "Sampling from f ~ GP(0, K)")

In [ ]:
xx = [1:4, 1:0.5:4, 1:0.2:4]
yy = [1, 0.5, 0.2]

plts = []
for (x, y) in zip(xx, yy)
    x1 = collect(x)
    n = Base.length(x)
    sample = rand(dist(gp, x1))
    sample_min = minimum(sample)
    sample_plot = scatter(x, sample, label = "",
            markershape = :utriangle, markercolor = :red,
            title = @sprintf("Sample, Step=%.1f", y), titlefontsize = 10)
    scatter!(x, fill(sample_min - 0.3, n),
        xlims = (0.5, 4.5),
        ylims = (sample_min - 0.5, maximum(sample) + 0.5),
        markershape = :x, markerstrokewidth = 2,
        label = "")
    push!(plts, sample_plot)
    push!(plts, heatmap(x1, x1, cov(gp, x1), 
            title = "Covariance matrix", titlefontsize = 10,
            aspect_ratio = :equal, yflip = true, color = cgrad([:white, :blue])))
end

Plots.plot(plts..., layout = (3, 2), size = [600, 600])

## Sampling from 2D Gaussian Process

In [ ]:
x1, x2 = 0:0.05:1, 0:0.05:1
n1, n2 = Base.length(x1), Base.length(x2)
n = n1 * n2
pt = permutedims(hcat([[i, j] for i in x1, j in x2]...));
sample = rand(dist(gp, pt), 2)

plts = []
for i in 1:2
    z = reshape(sample[:, i], n1, n2)
    push!(plts,
        surface(x1, x2, z, alpha = 0.8, camera = (40, 60),
        seriescolor = cgrad(ColorSchemes.jet1.colors), 
        linewidth = 2,
        framestyle = :grid,
        xlabel = "x1", ylabel = "x2", zlabel = "f"))
end
Plots.plot(plts..., layout = (1, 2), size = [700, 250])

## Kernel Types (Linear, Gaussian, Exponential, Periodic)

In [ ]:
xs = collect(range(-5, stop = 5, length = 100))
kernels = [LinearKernel(), GaussianKernel(1, 1),
           ExponentialKernel(1), PeriodicKernel(1.0, 0.5)]
plts = []

for k in kernels
    gp = GaussianProcess(k)
    p = Plots.plot(xs, rand(dist(gp, xs), 4), label = "", linewidth = 2, title = @sprintf("%s", k))
    push!(plts, p)
end
    
Plots.plot(plts..., layout = (2, 2), size = [800, 600])

## Linear combination

In [ ]:
xs = collect(range(-5, stop = 5, length = 100))
l_ker = 2.0 * LinearKernel() + 3.0 * PeriodicKernel(1.0, 0.5)

gp = GaussianProcess(l_ker)
Plots.plot(xs, rand(dist(gp, xs), 5), 
    label = "", size = [400, 300],
    linewidth = 2,
    title = "2 * LinearKernel() + 3 * PeriodicKernel(1.0, 0.5)", titlefontsize = 10)

## Matérn kernel

In [ ]:
xs = collect(range(-5, stop = 5, length = 100))
kernels = [MaternKernel(3/2, 1.), MaternKernel(5/2, 1.)]
plts = []

for k in kernels
    gp = GaussianProcess(k)
    p = Plots.plot(xs, rand(dist(gp, xs), 4), label = "", linewidth = 2, title = @sprintf("%s", k))
    push!(plts, p)
end
    
Plots.plot(plts..., layout = (1, 2), size = [800, 300])

## Gaussian Process Regression

In [ ]:
xs = [-0.5, 0.5, 1, 1.4, 3]
ys = [0.7, 1.8, 1.7, 2.3, 1]
gp = GaussianProcess(GaussianKernel(1.0, 0.4), 0.01)
xtrain = collect(range(-1, stop=3.5, length=100))
pred = gpr(gp, xtrain, xs, ys)
qt = mapslices(x -> quantile(x, [0.025, 0.925]), rand(pred, 10000), dims = 2)

Plots.plot(xtrain, qt[:, 1], fillrange = qt[:, 2], fillalpha = 0.3,
    label = "", linewidth = 0)
Plots.plot!(xtrain, mean(pred), label = "Mean", linewidth = 2, linestyle = :dash)
Plots.plot!(xtrain, rand(pred, 10), label = "", linewidth = 1, linealpha = 0.5)

scatter!(xs, ys, label = "", title = "Posterior distribution")

## Hyperparameter optimization

In [ ]:
xs = [-0.5, 0.5, 1, 1.4, 3]
ys = [0.7, 1.8, 1.7, 2.3, 1]

pc1 = ParamCalibrator(GaussianProcess(GaussianKernel(1, 1), 1), xs, ys)

In [ ]:
# optimization with gradient, GradientDescent
# we minimize -logp instead of maximize logp

lower = fill(-30.0, 3)
upper = fill(30.0, 3)

res = optimize(
    Optim.only_fg!((F, G, x) -> fg!(pc1, F, G, x)),
    lower, upper, [0.0, 0.0, 0.0], 
    Fminbox(GradientDescent()))

println(res)
pars = Optim.minimizer(res)
println("[theta1, theta2, theta3] = ", exp.(pars))

In [ ]:
# optimization with gradient, L-BFGS
lower = fill(-30.0, 3)
upper = fill(30.0, 3)

res = optimize(
    Optim.only_fg!((F, G, x) -> fg!(pc1, F, G, x)),
    lower, upper, [0.0, 0.0, 0.0], 
    Fminbox(LBFGS()))

println(res)
pars = Optim.minimizer(res)
println("[theta1, theta2, theta3] = ", exp.(pars))

In [ ]:
update!(pc1, pars)
xtrain = collect(range(-1, stop=3.5, length=100))
pred = gpr(pc1.gp, xtrain, pc1.xs, pc1.ys)
qt = mapslices(x -> quantile(x, [0.025, 0.975]), rand(pred, 10000), dims = 2)

Plots.plot(xtrain, qt[:, 1], fillrange = qt[:, 2], fillalpha = 0.3,
    label = "", linewidth = 0)
Plots.plot!(xtrain, mean(pred), label = "Mean", linewidth = 2, linestyle = :dash)

scatter!(xs, ys, label = "", title = "Posterior distribution")

In [ ]:
# log likelihood
logp(pc1)

In [ ]:
# Add more points

xs = [-0.5, 0.5, 1, 1.4, 3, 2.3, 2.5, 1.5, 1.1, 0.7]
ys = [0.7, 1.8, 1.7, 2.3, 1, 0, 0.2, 2, 2.4, 1.5]

pc2 = ParamCalibrator(GaussianProcess(GaussianKernel(1, 1), 1), xs, ys)
res = optimize(
    Optim.only_fg!((F, G, x) -> fg!(pc2, F, G, x)),
    lower, upper, [0.0, 0.0, 0.0], 
    Fminbox(LBFGS()))

println(res)
pars = Optim.minimizer(res)
println("[theta1, theta2, theta3] = ", exp.(pars))

update!(pc2, pars)
xtrain = collect(range(-1, stop=3.5, length=100))
pred = gpr(pc2.gp, xtrain, pc2.xs, pc2.ys)
qt = mapslices(x -> quantile(x, [0.025, 0.975]), rand(pred, 10000), dims = 2)

Plots.plot(xtrain, qt[:, 1], fillrange = qt[:, 2], fillalpha = 0.3,
    label = "", linewidth = 0)
Plots.plot!(xtrain, mean(pred), label = "Mean", linewidth = 2, linestyle = :dash)

scatter!(xs, ys, label = "", title = "Posterior distribution")

In [ ]:
# log likelihood
logp(pc2)

## Local solutions

In [ ]:
function f_logp(x, y)
    update!(pc1, [0, x, y])
    max(-20, logp(pc1)) # set floor of logp to be -20
end

theta2, theta3 = -2.5:0.2:12, -10:0.2:2.5
z = Surface((x, y) -> f_logp(x, y), theta2, theta3)
contourf(theta2, theta3, z, seriescolor = cgrad(ColorSchemes.jet1.colors), 
    levels = collect(-20:0.5:-2),
    xlabel = L"\log \theta_2", ylabel = L"\log \theta_3",
    guidefont=font(12))

In [ ]:
surface(theta2, theta3, z, seriescolor = cgrad(ColorSchemes.jet1.colors), camera = (30, 70))

In [ ]:
lower = fill(-30.0, 2)
upper = fill(30.0, 2)

function fg_2!(pc::ParamCalibrator, F, G, x)
    update!(pc, [0, x...])
    
    if G!= nothing
        y = exp.(x)
        d_tau = pc.k - y[2] .* Matrix{Float64}(I, pc.n_xs, pc.n_xs)
        d_sigma = d_tau ./ y[1] .* pc.distance_matrix
        d_eta = y[2] .* Matrix{Float64}(I, pc.n_xs, pc.n_xs)
        G[1] = -deriv(pc, d_sigma)
        G[2] = -deriv(pc, d_eta)
    end
    
    if F!= nothing
        value = -logp(pc)
        return value
    end

end

# starting from (0, 0)
res = optimize(
    Optim.only_fg!((F, G, x) -> fg_2!(pc1, F, G, x)),
    lower, upper, [0.0, 0.0], 
    Fminbox(LBFGS()))

println(res)
pars = Optim.minimizer(res)
println("logp:", logp(pc1))

update!(pc1, [0, pars...])
xtrain = collect(range(-1, stop=3.5, length=100))
pred = gpr(pc1.gp, xtrain, pc1.xs, pc1.ys)
qt = mapslices(x -> quantile(x, [0.025, 0.975]), rand(pred, 10000), dims = 2)

Plots.plot(xtrain, qt[:, 1], fillrange = qt[:, 2], fillalpha = 0.3,
    label = "", linewidth = 0)
Plots.plot!(xtrain, mean(pred), label = "Mean", linewidth = 2, linestyle = :dash)

scatter!(pc1.xs, pc1.ys, label = "", title = "Posterior distribution")

In [ ]:
# starting from (5, 0)
res = optimize(
    Optim.only_fg!((F, G, x) -> fg_2!(pc1, F, G, x)),
    lower, upper, [5.0, 0.0], 
    Fminbox(LBFGS()))

println(res)
pars = Optim.minimizer(res)
println("logp:", logp(pc1))

update!(pc1, [0, pars...])
xtrain = collect(range(-1, stop=3.5, length=100))
pred = gpr(pc1.gp, xtrain, pc1.xs, pc1.ys)
qt = mapslices(x -> quantile(x, [0.025, 0.975]), rand(pred, 10000), dims = 2)

Plots.plot(xtrain, qt[:, 1], fillrange = qt[:, 2], fillalpha = 0.3,
    label = "", linewidth = 0)
Plots.plot!(xtrain, mean(pred), label = "Mean", linewidth = 2, linestyle = :dash)

scatter!(pc1.xs, pc1.ys, label = "", title = "Posterior distribution")